# Vertex AI Vector Search - RAG Demo



### Prerequisites

This tutorial requires a Google Cloud project that is linked with a billing account. To create a new project, take a look at [this document](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) to create a project and setup a billing account for it.

### Choose the runtime environment

This tutorial can be run on either Google Colab or [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

- To use Colab: Click [this link](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-quickstart.ipynb) to open the tutorial in Colab.

- To use Workbench: If it is the first time to use Workbench in your Google Cloud project, open [the Workbench console](https://console.cloud.google.com/vertex-ai/workbench) and click ENABLE button to enable Notebooks API. The click [this link](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/embeddings/vector-search-quickstart.ipynb),  and select an existing notebook or create a new notebook.


### How much will this cost?

To go through this tutorial, it will cost roughly a few US dollars. The pricing of the Cloud services we will use in this tutorial are available in the following pages:

- [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/pricing#generative_ai_models)
- [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/pricing#matchingengine)
- [Vertex AI Workbench](https://cloud.google.com/vertex-ai/pricing#notebooks) if you use one

You can use the [Pricing Calculator](https://cloud.google.com/products/calculator) to generate a cost estimate based on your projected usage.

### **Warning: delete your objects after the tutorial**

In case you are using your own Cloud project, please make sure to delete all the Indexes and Index Endpoints (and the Workbench instance if you use one) after finishing this tutorial. Otherwise the remaining assets would incur unexpected costs.


## Setup

Before get started with the Vertex AI services, we need to setup the following.

* Install Python SDK
* Environment variables
* Authentication (Colab only)
* Enable APIs
* Set IAM permissions

### Install the Vertex AI SDK

Vertex AI APIs can be accessed with multiple ways including REST API and Python SDK. In this tutorial we will use the SDK.

In [ ]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 25.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

For Colab users: you will see "Your session crushed" message when restarting the kernel, but it is expected. Continue to the following cells.

In [15]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. In Colab or Colab Enterprise, you might see an error message that says "Your session crashed for an unknown reason." This is expected. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Environment variables

Sets environment variables. If asked, replace the following `[your-project-id]` with your project ID and run it.

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

Please set the project ID manually below


In [2]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "uk-con-gcp-sbx-ukat01-012822"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

### Authentication (Colab only)

If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [3]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set IAM permissions

Also, we need to add access permissions to the default service account for using those services.

- Go to [the IAM page](https://console.cloud.google.com/iam-admin/) in the Console
- Look for the principal for default compute service account. It should look like: `<project-number>-compute@developer.gserviceaccount.com`
- Click the edit button at right and click `ADD ANOTHER ROLE` to add `Vertex AI User` and `Service Usage Admin` roles to the account.


### Start

In [4]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai

In [5]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [6]:
# Get RagCorpus
# Configure embedding model we're using text-embedding-004.
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-004"
)


In [7]:
vector_search_index_name = "projects/387845474414/locations/us-central1/indexes/4735425057017298944"
vector_search_index_endpoint_name = "projects/387845474414/locations/us-central1/indexEndpoints/3200401271525736448"
display_name = "test_corpus"
description = "Corpus Description"
# Configure Vector DB
vector_db = rag.VertexVectorSearch(
    index=vector_search_index_name, index_endpoint=vector_search_index_endpoint_name
)

In [8]:
# corpora = rag.list_corpora()
# print(corpora)
corpus = rag.create_corpus(
    display_name=display_name,
    description=description,
    embedding_model_config=embedding_model_config,
    vector_db=vector_db
)

In [9]:
paths = ["gs://bank-product-rec"]  # Supports Google Cloud Storage and Google Drive Links

response = rag.import_files(
    corpus_name=corpus.name,
    paths=paths,
    chunk_size=120,  # Optional
    chunk_overlap=60,  # Optional
    max_embedding_requests_per_min=900,  # Optional
)
print(f"Imported {response.imported_rag_files_count} files.")

Imported 1 files.


In [10]:
# Enhance generation
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=corpus.name,  # Currently only 1 corpus is allowed.
                    # Optional: supply IDs from `rag.list_files()`.
                    # rag_file_ids=["rag-file-1", "rag-file-2", ...],
                )
            ],
            similarity_top_k=3,  # Optional
            vector_distance_threshold=0.5,  # Optional
        ),
    )
)

In [11]:
# Create a gemini-pro model instance
rag_model = GenerativeModel(
    # model_name="gemini-1.5-flash-001", tools=[rag_retrieval_tool]
    model_name="gemini-1.5-pro-002", tools=[rag_retrieval_tool]
)


In [12]:
# Generate response
response = rag_model.generate_content("recommend a saving's account for an adult with tax free interest")
print(response.text)

Consider a Cash ISA for tax-free interest on balances up to £20,000 with instant access, or a Fixed Rate ISA for tax-free interest on a lump sum for a fixed term of 1 or 2 years (minimum £1,000 deposit).



##**Clean Up**

In case you are using your own Cloud project, not a temporary project on Qwiklab, please make sure to delete all the Indexes and Index Endpoints after finishing this tutorial. Otherwise the remaining objects would incur unexpected costs.

In [ ]:
#Get list of corpora
corpora = rag.list_corpora()
print(corpora)

In [ ]:
# delete corpus
rag.delete_corpus(name=corpus_name)

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# delete index
my_index_id = ""  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

In [ ]:
# delete index endpoint
my_index_endpoint_id = ""  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)